In [1]:
import pandas as pd

df = pd.read_csv('../dados/jurisprudencia_selecionada_enunciados.csv', sep = '|')
df.head()

,COD,NUM_ENUNCIADO,COD_AREA,DESCR_AREA,COD_TEMA,DESCR_TEMA,COD_SUBTEMA,DESCR_SUBTEMA,COD_DOC_TRAMITAVEL_ENUNCIADO,TEXTO_ENUNCIADO,ACORDAO,TIPO_PROCESSO
0,1400,1236,50,Responsabilidade,488,Solidariedade,261,Benefício previdenciário,54995437,Não comprovada a participação do beneficiário ...,Acórdão 297/2016 - PL,Tomada de Contas Especial
1,1700,1534,46,Finanças Públicas,981,Exportação,983,Petróleo,55025587,A operação ficta de exportação de plataformas ...,Acórdão 366/2016 - PL,Solicitação do Congresso Nacional
2,5700,5314,50,Responsabilidade,203,Multa,1021,Dosimetria,55455370,"No âmbito do TCU, a dosimetria da pena tem com...",Acórdão 944/2016 - PL,Acompanhamento
3,284,40,45,Direito Processual,162,Princípio da independência das instâncias,481,Decisão judicial,54773746,O princípio da independência das instâncias pe...,Acórdão 30/2016 - PL,Tomada de Contas Especial
4,298,54,49,Pessoal,141,Sistema S,142,Nepotismo,54773402,É vedado aos dirigentes das entidades do Siste...,Acórdão 55/2016 - PL,Representação


In [2]:
df.shape

(13312, 12)

In [3]:
areas = df.groupby(['DESCR_AREA']).groups.keys()
areas

dict_keys(['Competência do TCU', 'Contrato Administrativo', 'Convênio', 'Desestatização', 'Direito Processual', 'Finanças Públicas', 'Gestão Administrativa', 'Licitação', 'Pessoal', 'Responsabilidade'])

In [4]:
from sklearn.preprocessing import LabelBinarizer

lbArea = LabelBinarizer()
lbArea.fit([x for x in areas])
lbArea.classes_

array(['Competência do TCU', 'Contrato Administrativo', 'Convênio',
       'Desestatização', 'Direito Processual', 'Finanças Públicas',
       'Gestão Administrativa', 'Licitação', 'Pessoal',
       'Responsabilidade'], dtype='<U23')

In [5]:
y = lbArea.transform(df['DESCR_AREA'])
y.shape

(13312, 10)

In [6]:
from keras.preprocessing.text import Tokenizer
import numpy as np

vocabulario = 20000
limite_texto = 200
dim_vetor = 100

tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['TEXTO_ENUNCIADO'])
vocabulario = len(tokenizer.word_index) + 1

sequences = tokenizer.texts_to_sequences(df['TEXTO_ENUNCIADO'])

Using TensorFlow backend.


In [7]:
from keras.preprocessing.sequence import pad_sequences

x = pad_sequences(sequences, maxlen=limite_texto)

print('Shape of data tensor:', x.shape)

Shape of data tensor: (13312, 200)


In [8]:
alternativas = [None, 'sem pré-treino', 'NILC fixo', 'NILC variável', 'Acordãos fixo', 'Acordãos variável']

In [9]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

print('Montando embeddings NILC:')
model_nilc = KeyedVectors.load_word2vec_format('../externos/model-cbow.txt')
embedding_matrix_nilc = np.zeros((vocabulario, dim_vetor))
ok = 0
for word, i in tokenizer.word_index.items():
    if word in model_nilc:
        embedding_matrix_nilc[i] = model_nilc[word]
        ok += 1
print('\tVocabulario:', i, ' - encontrados no modelo:', ok, '=', ok * 100. / i)

print('Montando embeddings Acordaos:')
model_acordaos = Word2Vec.load('../vocabularios/modelo-acordaos.w2v')
embedding_matrix_acordaos = np.zeros((vocabulario, dim_vetor))
ok = 0
for word, i in tokenizer.word_index.items():
    if word in model_acordaos.wv:
        embedding_matrix_acordaos[i] = model_acordaos.wv[word]
        ok += 1
print('\tVocabulario:', i, ' - encontrados no modelo:', ok, '=', ok * 100. / i)

embeddings = [None, None, embedding_matrix_nilc, embedding_matrix_nilc, embedding_matrix_acordaos, embedding_matrix_acordaos]

Montando embeddings NILC:


/home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


	Vocabulario: 15387  - encontrados no modelo: 13758 = 89.41314096315071
Montando embeddings Acordaos:
	Vocabulario: 15387  - encontrados no modelo: 14572 = 94.70332098524729


In [10]:
colunas_scores = list(lbArea.classes_)
colunas_scores.extend(['accuracy', 'macro avg', 'weighted avg'])

In [11]:
def gera_embedding(alt, model, vocabulario, dim_vetor, limite_texto):
        if alt == 1:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True))
        elif alt == 2:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=False, weights=[embedding_matrix_nilc]))
        elif alt == 3:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True,  weights=[embedding_matrix_nilc]))
        elif alt == 4:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=False, weights=[embedding_matrix_acordaos]))
        elif alt == 5:
            model.add(Embedding(vocabulario, dim_vetor, input_length=limite_texto, trainable=True,  weights=[embedding_matrix_acordaos]))

In [12]:
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.optimizers import RMSprop
from keras.callbacks import ModelCheckpoint

print('Realizando treinamentos:')
df_medias = pd.DataFrame()
for alt in range(1, 6):
    str_alt = str(alt)
    fold = 0
    df_scores = pd.DataFrame()
    for train_index, val_index in KFold(n_splits=10, random_state=42, shuffle=True).split(df):
        str_fold = 'Fold ' + str(fold)
        print()
        print('Alternativa ' + alternativas[alt], '-', str_fold + ':')

        df_train = df.loc[train_index]
        df_val = df.loc[val_index]

        sequences_train = tokenizer.texts_to_sequences(df_train['TEXTO_ENUNCIADO'])
        sequences_val = tokenizer.texts_to_sequences(df_val['TEXTO_ENUNCIADO'])

        x_train = pad_sequences(sequences_train, maxlen=limite_texto)
        x_val = pad_sequences(sequences_val, maxlen=limite_texto)

        y_train = lbArea.transform(df_train['DESCR_AREA'])
        y_val = lbArea.transform(df_val['DESCR_AREA'])

        model = Sequential()
        gera_embedding(alt, model, vocabulario, dim_vetor, limite_texto)
        model.add(GRU(256, dropout=0.2, recurrent_dropout=0.2))
        model.add(Dense(y.shape[1], activation='softmax'))
        model.compile(loss='categorical_crossentropy', optimizer=RMSprop(),  metrics=['categorical_accuracy'])

        checkpoint_filename = 'weights' + str_alt + '-' + str(fold) + '.hdf5'
        checkpointer = ModelCheckpoint(filepath=checkpoint_filename, monitor='val_categorical_accuracy', verbose=1, save_best_only=True)
        model.fit(x_train, y_train, epochs=20, batch_size=32, validation_data=(x_val, y_val), verbose=1, shuffle=False, callbacks=[checkpointer])

        print('Evaluating best model and registering score:')
        model.load_weights(checkpoint_filename)
        y_val_pred = model.predict_classes(x_val, verbose=1)
        y_val_i = [list(x).index(1) for x in y_val]
        report = classification_report(y_val_i, y_val_pred, target_names=lbArea.classes_, output_dict = True)
        for col in colunas_scores:
            if col == 'accuracy':
                f = report[col]
            else:
                f = report[col]['f1-score']
            df_scores.loc[str_fold,col] = f
        fold += 1
    df_medias[alternativas[alt] + ' mean'] = df_scores.mean()
    df_medias[alternativas[alt] + ' std'] = df_scores.std()
df_medias.T

Realizando treinamentos:

Alternativa sem pré-treino - Fold 0:


W1012 19:01:49.591818 140466233853760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W1012 19:01:49.660044 140466233853760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W1012 19:01:49.674020 140466233853760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W1012 19:01:50.210307 140466233853760 deprecation_wrapper.py:119] From /home/leonardo/anaconda3/envs/gpu/lib/python3.7/site-packages/keras/backend/tensorflow_backend.py:133: The name tf.placeholder_with_default is d

Train on 11980 samples, validate on 1332 samples
Epoch 1/20
11980/11980 [==============================] - 83s 7ms/step - loss: 1.5695 - categorical_accuracy: 0.4876 - val_loss: 1.3462 - val_categorical_accuracy: 0.5511

Epoch 00001: val_categorical_accuracy improved from -inf to 0.55105, saving model to weights1-0.hdf5
Epoch 2/20
11980/11980 [==============================] - 71s 6ms/step - loss: 0.9764 - categorical_accuracy: 0.6853 - val_loss: 0.9776 - val_categorical_accuracy: 0.6667

Epoch 00002: val_categorical_accuracy improved from 0.55105 to 0.66667, saving model to weights1-0.hdf5
Epoch 3/20
11980/11980 [==============================] - 71s 6ms/step - loss: 0.6902 - categorical_accuracy: 0.7702 - val_loss: 0.7709 - val_categorical_accuracy: 0.7545

Epoch 00003: val_categorical_accuracy improved from 0.66667 to 0.75450, saving model to weights1-0.hdf5
Epoch 4/20
11980/11980 [==============================] - 70s 6ms/step - loss: 0.5355 - categorical_accuracy: 0.8230 - val_los

11980/11980 [==============================] - 68s 6ms/step - loss: 0.1255 - categorical_accuracy: 0.9611 - val_loss: 0.5526 - val_categorical_accuracy: 0.8544

Epoch 00011: val_categorical_accuracy did not improve from 0.86261
Epoch 12/20
11980/11980 [==============================] - 68s 6ms/step - loss: 0.1012 - categorical_accuracy: 0.9700 - val_loss: 0.5778 - val_categorical_accuracy: 0.8476

Epoch 00012: val_categorical_accuracy did not improve from 0.86261
Epoch 13/20
11980/11980 [==============================] - 68s 6ms/step - loss: 0.0818 - categorical_accuracy: 0.9753 - val_loss: 0.5351 - val_categorical_accuracy: 0.8604

Epoch 00013: val_categorical_accuracy did not improve from 0.86261
Epoch 14/20
11980/11980 [==============================] - 68s 6ms/step - loss: 0.0678 - categorical_accuracy: 0.9808 - val_loss: 0.6080 - val_categorical_accuracy: 0.8559

Epoch 00014: val_categorical_accuracy did not improve from 0.86261
Epoch 15/20
11980/11980 [===========================

Epoch 2/20
11981/11981 [==============================] - 68s 6ms/step - loss: 0.9983 - categorical_accuracy: 0.6754 - val_loss: 1.0073 - val_categorical_accuracy: 0.6574

Epoch 00002: val_categorical_accuracy improved from 0.56950 to 0.65740, saving model to weights1-3.hdf5
Epoch 3/20
11981/11981 [==============================] - 68s 6ms/step - loss: 0.7036 - categorical_accuracy: 0.7701 - val_loss: 0.7608 - val_categorical_accuracy: 0.7400

Epoch 00003: val_categorical_accuracy improved from 0.65740 to 0.74005, saving model to weights1-3.hdf5
Epoch 4/20
11981/11981 [==============================] - 68s 6ms/step - loss: 0.5128 - categorical_accuracy: 0.8378 - val_loss: 0.5915 - val_categorical_accuracy: 0.8092

Epoch 00004: val_categorical_accuracy improved from 0.74005 to 0.80917, saving model to weights1-3.hdf5
Epoch 5/20
11981/11981 [==============================] - 68s 6ms/step - loss: 0.4143 - categorical_accuracy: 0.8703 - val_loss: 0.5100 - val_categorical_accuracy: 0.8385



...

11981/11981 [==============================] - 70s 6ms/step - loss: 0.1209 - categorical_accuracy: 0.9589 - val_loss: 0.4070 - val_categorical_accuracy: 0.8805

Epoch 00009: val_categorical_accuracy did not improve from 0.88730
Epoch 10/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.1058 - categorical_accuracy: 0.9648 - val_loss: 0.3969 - val_categorical_accuracy: 0.8828

Epoch 00010: val_categorical_accuracy did not improve from 0.88730
Epoch 11/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0932 - categorical_accuracy: 0.9678 - val_loss: 0.4045 - val_categorical_accuracy: 0.8903

Epoch 00011: val_categorical_accuracy improved from 0.88730 to 0.89031, saving model to weights5-5.hdf5
Epoch 12/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0729 - categorical_accuracy: 0.9765 - val_loss: 0.4338 - val_categorical_accuracy: 0.8933

Epoch 00012: val_categorical_accuracy improved from 0.89031 to 0.89331, saving mod


Epoch 00020: val_categorical_accuracy did not improve from 0.89782
Evaluating best model and registering score:
1331/1331 [==============================] - 5s 3ms/step

Alternativa Acordãos variável - Fold 7:
Train on 11981 samples, validate on 1331 samples
Epoch 1/20
11981/11981 [==============================] - 76s 6ms/step - loss: 0.9494 - categorical_accuracy: 0.6897 - val_loss: 0.6337 - val_categorical_accuracy: 0.7904

Epoch 00001: val_categorical_accuracy improved from -inf to 0.79038, saving model to weights5-7.hdf5
Epoch 2/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.5171 - categorical_accuracy: 0.8258 - val_loss: 0.4571 - val_categorical_accuracy: 0.8467

Epoch 00002: val_categorical_accuracy improved from 0.79038 to 0.84673, saving model to weights5-7.hdf5
Epoch 3/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.3976 - categorical_accuracy: 0.8675 - val_loss: 0.4252 - val_categorical_accuracy: 0.8595

Epoch 00003: val_ca

11981/11981 [==============================] - 70s 6ms/step - loss: 0.0986 - categorical_accuracy: 0.9664 - val_loss: 0.4243 - val_categorical_accuracy: 0.8843

Epoch 00010: val_categorical_accuracy did not improve from 0.88505
Epoch 11/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0888 - categorical_accuracy: 0.9707 - val_loss: 0.4448 - val_categorical_accuracy: 0.8798

Epoch 00011: val_categorical_accuracy did not improve from 0.88505
Epoch 12/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0834 - categorical_accuracy: 0.9738 - val_loss: 0.4718 - val_categorical_accuracy: 0.8760

Epoch 00012: val_categorical_accuracy did not improve from 0.88505
Epoch 13/20
11981/11981 [==============================] - 70s 6ms/step - loss: 0.0728 - categorical_accuracy: 0.9757 - val_loss: 0.4266 - val_categorical_accuracy: 0.8896

Epoch 00013: val_categorical_accuracy improved from 0.88505 to 0.88956, saving model to weights5-8.hdf5
Epoch 14/20
119

1331/1331 [==============================] - 5s 4ms/step


,Competência do TCU,Contrato Administrativo,Convênio,Desestatização,Direito Processual,Finanças Públicas,Gestão Administrativa,Licitação,Pessoal,Responsabilidade,accuracy,macro avg,weighted avg
sem pré-treino mean,0.798218,0.759254,0.719760,0.481583,0.879672,0.709347,0.599682,0.898277,0.959358,0.852032,0.865986,0.765718,0.863602
sem pré-treino std,0.040992,0.029363,0.023817,0.120425,0.007820,0.089558,0.099139,0.015717,0.006882,0.012400,0.004898,0.025613,0.004977
NILC fixo mean,0.818819,0.737703,0.707960,0.562877,0.849191,0.648444,0.605647,0.885713,0.942283,0.828468,0.848634,0.758710,0.846768
NILC fixo std,0.053109,0.034749,0.039108,0.169379,0.009764,0.065940,0.089725,0.010886,0.009341,0.016883,0.009400,0.028640,0.009698
NILC variável mean,0.830709,0.813408,0.764688,0.723404,0.895083,0.757757,0.703845,0.921653,0.964590,0.881249,0.891827,0.825639,0.890862
NILC variável std,0.049933,0.028637,0.024634,0.143204,0.013851,0.033463,0.074025,0.013856,0.007803,0.012025,0.002703,0.020483,0.002915
Acordãos fixo mean,0.844262,0.816256,0.786990,0.746612,0.898457,0.747484,0.680443,0.922223,0.971261,0.883894,0.895734,0.829788,0.894914
Acordãos fixo std,0.045620,0.032449,0.034925,0.108680,0.015582,0.033849,0.071216,0.013486,0.006555,0.018151,0.007541,0.019492,0.007787
Acordãos variável mean,0.824382,0.828329,0.799035,0.783671,0.901800,0.790979,0.712000,0.922546,0.970619,0.889691,0.900016,0.842305,0.899260
Acordãos variável std,0.041929,0.028223,0.036238,0.074757,0.013480,0.051590,0.053897,0.017688,0.006209,0.016206,0.006298,0.016962,0.006349


In [13]:
df_medias.T.to_csv('scores_embeddings_gru.csv', encoding = 'Latin1')